# Import Libraries

In [1]:
import os
import sys
import utils
# import models
import torch
from tqdm import tqdm_notebook
from torch.utils.data import DataLoader
from torchvision import datasets
from gan import *
import torchvision.transforms as transforms
import torch.nn as nn
    

import os
import time
import torch
import models
import utils

from sklearn.metrics import confusion_matrix
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
from torchvision.utils import save_image, make_grid

import seaborn as sns  # For visualizing the confusion matrix
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

 
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

from utils import *
import pdb

 
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

from utils import *
import pdb

import os
import time
import torch
import models
import utils
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.cluster import AgglomerativeClustering


from sklearn.metrics import confusion_matrix
from torch.autograd import Variable
from torch.autograd import Variable
from torchvision.utils import save_image, make_grid

import seaborn as sns  # For visualizing the confusion matrix
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

from scipy.cluster import hierarchy

import pdb

/home/roy206/.conda/envs/cent7/2020.11-py38/NWRGAE/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Models (28 x 28)

In [2]:

class Generator(nn.Module):
    def __init__(self, opt):
        super(Generator, self).__init__()
        self.opt = opt
        self.architecture = self.opt.architecture

        
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        modules = nn.ModuleList()
        
        for _ in range(opt.n_paths_G):
            modules.append(
                nn.Sequential(
                *block(opt.latent_dim,2,normalize=False),
                *block(2,128),
                *block(128,256),
                *block(256,512),
                *block(512,1024),
                nn.Linear(1024, int(np.prod(opt.img_shape))),
                nn.Tanh()
            ))
        self.paths = modules

    def forward(self, z):
        img = []
        for path in self.paths:
            img.append(path(z).view(img.size(0), *self.opt.img_shape))
        img = torch.cat(img, dim=0)
        return img

class Discriminator(nn.Module):
    def __init__(self, opt):
        super(Discriminator, self).__init__()

        self.fc1 = nn.Linear(int(np.prod(opt.img_shape)), 512)
        self.lr1 = nn.LeakyReLU(0.2, inplace=True)
        self.fc2 = nn.Linear(512, 256)
        self.lr2 = nn.LeakyReLU(0.2, inplace=True)
        
        encoder_layers = []

        encoder_layers += [self.fc1]
        encoder_layers += [self.lr1]
        encoder_layers += [self.fc2]
        encoder_layers += [self.lr2]
        
        self.encoder_layers = nn.Sequential(*encoder_layers)
        
        modules = nn.ModuleList()
        
        modules.append(nn.Sequential(
            nn.Linear(256, 1),
            nn.Sigmoid(),
                ))
        modules.append(nn.Sequential(
            nn.Linear(256, opt.n_paths_G),
                ))
        self.paths = modules
    
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        # img_flat = self.lr2(self.fc2(self.lr1(self.fc1(img_flat))))
        img_flat = self.encoder_layers(img_flat)
        
        validity = self.paths[0](img_flat)
        classifier = F.log_softmax(self.paths[1](img_flat), dim=1)
        
        return validity, classifier



# Metrics

In [3]:

def clustering_acc(y_true, y_pred):
    """
    Calculate the clustering accuracy for the case where the number of clusters
    might be different from the number of true classes. It uses the Hungarian algorithm.

    Args:
    y_true (array-like): True labels.
    y_pred (array-like): Predicted cluster labels.

    Returns:
    float: Clustering accuracy.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    return sum([w[row, col] for row, col in zip(row_ind, col_ind)]) / y_pred.size


def get_cluster(dataloader, discriminator, soft_preds=False, noise_intensity=0):
    #cuda = True if torch.cuda.is_available() else False
    #Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    all_data = []
    all_idxs = []
    all_preds = []
    with torch.no_grad():
        for i, (x,y) in enumerate(dataloader):
            x = x.to(device)
            x = add_noise(x, noise_intensity)
            y = y.to(device)
            d_out, c_out = discriminator(x)
            all_preds.append(c_out)
            all_idxs.append(y)
    all_idxs = torch.cat(all_idxs, axis=0)
    all_preds = torch.cat(all_preds, axis=0)
    if soft_preds:
        return all_idxs, all_preds
    else:
        return all_idxs, torch.argmax(all_preds, dim=-1)



from scipy.cluster.hierarchy import dendrogram
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    

def add_noise(tensor, normal_std_scale):
    if (normal_std_scale > 0):
        return tensor + (tensor*torch.randn_like(tensor)*normal_std_scale)
    else:
        return tensor

# Train

In [4]:

def train(opt):
    generator = models.Generator(opt)
    discriminator = models.Discriminator(opt)
    img_shape = opt.img_shape

    cuda = torch.cuda.is_available()

    adversarial_loss = torch.nn.BCELoss()

    if cuda:
        generator.cuda()
        discriminator.cuda()
        adversarial_loss.cuda()

    # Configure data loader
    os.makedirs("data/fmnist", exist_ok=True)
    dataloader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(
            "data/fmnist",
            train=True,
            download=True,
            transform=transforms.Compose(
                [transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
            ),
        ),
        batch_size=opt.batch_size,
        shuffle=True,
    )
    
    Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

    for epoch in range(opt.n_epochs):

        start = time.time()
        all_targets = []
        all_predictions = []
        
        epoch_start = 0
        
        g_loss_epoch = 0
        d_loss_epoch = 0
        c_loss_1_epoch = 0
        c_loss_2_epoch = 0
        
        correct_predictions = 0
        total_predictions = 0

        num_batches = len(dataloader)
        
        for i, (imgs, _) in enumerate(dataloader):
            
            valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
            fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

            real_imgs = Variable(imgs.type(Tensor))

            # -----------------------------------------------------
            #  Train Generator 
            # -----------------------------------------------------

            optimizer_G.zero_grad()

            z = Variable(Tensor(np.random.normal(0, 1, (opt.batch_size_g, opt.latent_dim))))
            g_loss = 0 
            c_loss_1 = 0 
            
            for k in range(opt.n_paths_G):

                gen_imgs = generator.paths[k](z)
                validity, classifier = discriminator(gen_imgs)
                g_loss += adversarial_loss(validity, valid)

                target = Variable(Tensor(imgs.size(0)).fill_(k), requires_grad=False)
                target = target.type(torch.cuda.LongTensor)
                
                c_loss_1 += F.nll_loss(classifier, target) * opt.classifier_para_g

                _, predicted = torch.max(classifier.data, 1)
                total_predictions += target.size(0)
                correct_predictions += (predicted == target).sum().item()
        
                # Collect all targets and predictions for confusion matrix
                all_targets.extend(target.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())
            
            g_loss_epoch += g_loss
            c_loss_1_epoch += c_loss_1       
            g_loss = g_loss + c_loss_1
            
            g_loss.backward()
            optimizer_G.step()
            
            # ------------------------------------------------------------------------
            #  Train Discriminator and Classifier (WE CALL THIS STAGE PHASE 2)
            # ------------------------------------------------------------------------

            optimizer_D.zero_grad()

            d_loss = 0 #adversarial loss for discrimination to be applied to the discriminator
            c_loss_2 = 0 #classification loss to be applied to the classifier
            
            #loss of the discriminator with real images
            validity, classifier = discriminator(real_imgs)
            real_loss = adversarial_loss(validity, valid)
            
            temp = [] #variable to store images for plot
            
            #again we iterate over each of the generators
            for k in range(opt.n_paths_G):

                # generates a batch of images with generator k
                gen_imgs = generator.paths[k](z).view(imgs.shape[0], *img_shape)
                temp.append(gen_imgs[0:10, :])

                #again we measure the outputs of the discriminator and the classifier 
                validity, classifier = discriminator(gen_imgs.detach())
                
                #loss of the discriminator with fake images
                fake_loss = adversarial_loss(validity, fake)
                
                #sums up the fake and true losses
                d_loss += (real_loss + fake_loss) / 2

                #again we calculate the loss of the classifier. Note that it is calculated...
                #...in exactly the same way as before, but only after the...
                #...generator has already had its weights updated in PHASE 1.
                #reminder: opt.classifier_para is just a multiplicative constant of the loss
                target = Variable(Tensor(imgs.size(0)).fill_(k), requires_grad=False)
                target = target.type(torch.cuda.LongTensor)
                c_loss_2 += F.nll_loss(classifier, target) * opt.classifier_para
            
            #images for plot
            plot_imgs = torch.cat(temp, dim=0)

            #again, we accumulate the losses to visualize the total for the epoch
            d_loss_epoch += d_loss
            c_loss_2_epoch += c_loss_2
            
            #we add both losses into the same variable 
            d_loss = d_loss + c_loss_2
            #jointly apply the gradients to the layers of the discriminator and classifier
            d_loss.backward()
            optimizer_D.step()
        
        #printing training information
        interval = time.time() - start
        
        if (epoch < 20 and epoch % 5 == 0) or (epoch > 20 and epoch % 20 == 0):
            
            # print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [C loss 1: %f] [C loss 2: %f] \t Time Interv: %f"
            #        % (epoch, opt.n_epochs, i, num_batches, d_loss_epoch.item()/num_batches, g_loss_epoch.item()/num_batches, 
            #           c_loss_1_epoch.item()/num_batches, c_loss_2_epoch.item()/num_batches, interval))

            classification_accuracy = 100.0 * correct_predictions / total_predictions

            print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [C loss 1: %f] [C loss 2: %f] [Acc: %.2f%%] \t Time Interv: %f"
                % (epoch, opt.n_epochs, i, num_batches, d_loss_epoch.item()/num_batches, g_loss_epoch.item()/num_batches, 
                    c_loss_1_epoch.item()/num_batches, c_loss_2_epoch.item()/num_batches, classification_accuracy, interval))

            utils.show(make_grid(plot_imgs[:opt.n_paths_G*10].cpu(), nrow=10, normalize=True), opt)
            plt.show()
            
            # Compute and print confusion matrix at the end of each epoch
            cm = confusion_matrix(all_targets, all_predictions)/(opt.batch_size_g*len(dataloader))
            cm_rounded = np.around(cm, decimals=2)
            fig, ax = plt.subplots(figsize=(opt.n_paths_G//2, opt.n_paths_G//2))
            # sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap='Blues')
            sns.heatmap(cm_rounded, annot=True, fmt=".2f", linewidths=.5, square = True, cmap = 'viridis')
            ax.set_xlabel('Predicted Labels')
            ax.set_ylabel('True Labels')
            ax.set_title('Confusion Matrix')
            plt.show()


# Utils

In [5]:

def parse_args():

    
    parser = argparse.ArgumentParser()

    # parser.add_argument('-f')
    parser.add_argument('--n_epochs', type=int, default=500, help='number of epochs')
    parser.add_argument('--batch_size', type=int, default=100, help='batch size')
    parser.add_argument('--batch_size_g', type=int, default=100, help='generator batch size')
    parser.add_argument('--b1', type=float, default=0.5, help='adam: decay of first order momentum')
    parser.add_argument('--b2', type=float, default=0.999, help='adam: decay of second order momentum')
    parser.add_argument('--n_cpu', type=int, default=12, help='number of cpu threads to use during batch generation')
    parser.add_argument('--latent_dim', type=int, default=100, help='dimensionality of the latent space')
    parser.add_argument('--sample_interval', type=int, default=1, help='interval between image samples')
    parser.add_argument('--classifier_para', type=float, default=1.0, help='classifier loss weight')
    parser.add_argument('--classifier_para_g', type=float, default=1.0, help="generator's classification loss weight")
    parser.add_argument('--dataset', type=str, default='fmnist', help='dataset name') #choices=['fmnist', 'mnist', 'cifar10']
    parser.add_argument('--architecture', default='mlp', help='architecture for the model')
    parser.add_argument('--logs_path', default='logs_40', help='where to save logs')
    parser.add_argument('--init_noise', type=float, default=1.0, help='Initial stdd of gaussian noise applied to input of discriminator')
    parser.add_argument('--noise_decay', type=float, default=0.01, help='How much is subtracted of of gaussian stdd noise at each epoch')
    
    #generator parameters
    parser.add_argument('--n_paths_G', type=int, default=40, help='number of generators')
    parser.add_argument('--lr_g', type=float, default=0.001, help='learning rate for gen')
    
    parser.add_argument('--nf_g', type=int, default=128,help='Number of feature maps for generator.')
    # parser.add_argument('--norm_g', type=str, default='layer_norm', choices=['layer_norm', 'batch_norm', 'no_norm'], help='Type of normalization layer used for generator')
    parser.add_argument('--no_conv_g', type=int, default=2, choices=[2], help='Number of conv layers for gen')
    
    #discriminator/classifier parameters
    parser.add_argument('--lr_d', type=float, default=0.001, help='learning rate for disc')
    parser.add_argument('--lr_c', type=float, default=0.001, help='learning rate for clasf')
    parser.add_argument('--nf_d', type=int, default=128, help='Number of feature maps for discriminator.')
    parser.add_argument('--norm_d', type=str, default='layer_norm', choices=['layer_norm', 'batch_norm', 'no_norm'], help='Type of normalization layer used for discriminator')
    parser.add_argument('--no_conv_d', type=int, default=3, choices=[3], help='Number of conv layers for discriminator')
    
    
    parser.set_defaults(shared_features_across_ref=False)


    args = parser.parse_args()

    if args.dataset == 'cifar10':
        args.img_size = 32
        args.channels = 3
    else:  
        args.img_size = 28
        args.channels = 1

    args.img_shape = (args.channels, args.img_size, args.img_size)

    return args


def show(img, opt):
    npimg = img.detach().numpy()
    dim = max(5, opt.n_paths_G//2)
    plt.figure(figsize = (dim, dim))
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

# Main.py

In [ ]:
    
def main():
    # opt = utils.arg_parser_subst(sys.argv)
    opt = utils.parse_args()
    opt.dataset = "fmnist"
    opt.logs_path = "logs_path_40"
    opt.n_paths_G = 40
    # print(opt.dataset)

    if opt.dataset == "fmnist":
        os.makedirs("data/fmnist", exist_ok=True)
        dataloader = torch.utils.data.DataLoader(
            datasets.FashionMNIST(
                "data/fmnist",
                train=True,
                download=True,
                transform=transforms.Compose(
                    [transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
                ),
            ),
            batch_size=opt.batch_size,
            shuffle=True,
        )
        if("cnn" in opt.architecture):
            import models_cnn_28 as models
        else:
            import models
        

    if opt.dataset == "mnist":
        os.makedirs("data/mnist", exist_ok=True)
        dataloader = torch.utils.data.DataLoader(
            datasets.MNIST(
                "data/mnist",
                train=True,
                download=True,
                transform=transforms.Compose(
                    [transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
                ),
            ),
            batch_size=opt.batch_size,
            shuffle=True,
        )
        if("cnn" in opt.architecture):
            import models_cnn_28 as models
        else:
            import models
        
        
    elif opt.dataset == "cifar10":
        os.makedirs("data/CIFAR10", exist_ok=True)
        dataloader = torch.utils.data.DataLoader(
            datasets.CIFAR10('data/CIFAR10', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize([.5],[.5])
                   ])
            ),
            batch_size=opt.batch_size, 
            shuffle=True)
        if("cnn" in opt.architecture):
            import models_cnn_32 as models
        else:
            import models

    generator = Generator(opt) #ok 
    discriminator = Discriminator(opt) # ok
    
    # print("Generator: ", generator)
    # print("Discriminator/classifier: ", discriminator)
    
    #optimizer G
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr_g, betas=(opt.b1, opt.b2))
    
    #optimizer D
    optimizer_D = torch.optim.Adam(list(discriminator.encoder_layers.parameters()) + list(discriminator.paths[0].parameters()), lr=opt.lr_d, betas=(opt.b1, opt.b2))
    # optimizer_D.add_param_group(discriminator.paths[0].parameters())
    
    #optimizer C
    optimizer_C = torch.optim.Adam(discriminator.paths[1].parameters(), lr=opt.lr_c, betas=(opt.b1, opt.b2))
    gan = Gan(generator, discriminator, optimizer_G, optimizer_D, optimizer_C, opt)

    gan.train(dataloader)

import sys
sys.argv=['']
del sys
    
if __name__ == "__main__":
    main()

[Epoch 0/500] [D loss: 26.325101] [G loss: 31.987572] [C loss 1: 8.523518] [C loss 2: 8.502166] [Acc: 98.43%] 	 Time Interv: 329.632448
Clustering metrics:
NMI:0.368485329766552
ACC:0.2737833333333333
40
[0.368485329766552, 0.3542245778394509, 0.3523124675732253, 0.3497701384699673, 0.3411890106629037, 0.3415256559494391, 0.341873393048393, 0.32503942782046086, 0.31466374018098575, 0.30976744037273507, 0.3094598608608991, 0.3071156225828966, 0.3052425779211719, 0.28674951207779853, 0.28068496773707896, 0.28019833126607513, 0.27502546339771444, 0.26667686041914856, 0.26697114231554603, 0.2346939846252207, 0.21004930101029548, 0.19835477091476922, 0.19562657111650042, 0.1952577245015422, 0.18899913066706006, 0.1729177188351592, 0.16955726466346321, 0.16035646458438693, 0.11014564105506795, 0.10507043726275217, 0.10101401688702842]
[0.2737833333333333, 0.27386666666666665, 0.2739, 0.2874, 0.2817, 0.2817, 0.2894833333333333, 0.26005, 0.26005, 0.2601, 0.26011666666666666, 0.2601166666666666